In [1]:
import cv2
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
with_mask = np.load('with_mask500_new.npy')
without_mask = np.load('without_mask500_new.npy')

In [3]:
without_mask.shape


(500, 50, 50, 3)

In [4]:
with_mask.shape

(500, 50, 50, 3)

In [5]:
with_mask = with_mask.reshape(500, 50 * 50 * 3)
without_mask = without_mask.reshape(500,50 * 50 * 3)

In [6]:
X = np.r_[with_mask,without_mask]

In [7]:
label = np.zeros(X.shape[0])

In [8]:
label[500:] = 1.0

In [9]:
x_train, x_test ,y_train, y_test = train_test_split(X,label,test_size=0.25)

In [10]:
x_train[0]

array([255,   0, 255, ..., 231,   6, 228], dtype=uint8)

In [11]:
x_train.shape

(750, 7500)

In [12]:
x_test.shape

(250, 7500)

In [13]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
x_train = pca.fit_transform(x_train)

In [14]:
x_train[0]

array([-2505.26843712,   235.25575879,  2969.45852944])

In [15]:
svm = SVC()
svm.fit(x_train,y_train)

SVC()

In [16]:
x_test = pca.transform(x_test)
y_pred = svm.predict(x_test)

In [17]:
accuracy_score(y_test , y_pred)

0.992

In [18]:
names = {0:'Mask',1:'No Mask'}

In [21]:
haar_data = cv2.CascadeClassifier('data.xml')
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag,img = capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,255),4)
            face = img[y:y+h,x:x+w,:]
            face = cv2.resize(face,(50,50))
            face = face.reshape(1,-1)
            face = pca.transform(face)
            pred = svm.predict(face)[0]
            n = names[int(pred)]
            cv2.putText(img,n,(x,y),font,1,(244,250,250),2)
            print(n)
        cv2.imshow('result',img)
        if cv2.waitKey(2) == 27 :
            break
capture.release()
cv2.destroyAllWindows()

No Mask
No Mask
No Mask
No Mask
No Mask
Mask
No Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
No Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
Mask
No Mask
No Mask
No Mask
Mask
No Mask
No Mask
Mask
Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
No Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
Mask
M

In [20]:
import pickle
filename = './finalized_model500_new.sav'
pickle.dump(svm, open(filename, 'wb'))